In [5]:
#equal number in each slice for SIR
from tabulate import tabulate
import numpy as np
from scipy.linalg import solve_triangular
def sir_modified(X, y, num_slices, K):
    X = X - np.mean(X, axis=0)
    #reduced QR decomposition
    Q, R = np.linalg.qr(X)
    n_samples, n_features = X.shape
    Z = np.sqrt(n_samples) * Q
    
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    Z_sorted = Z[sorted_indices]
    y_sorted = y[sorted_indices]

    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []

    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((Z_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    
    # Step 3: Compute the means of the predictors within each slice
    Z_means = np.array([np.mean(slice_Z, axis=0) for slice_Z, _ in slices])
    
    # Step 4: Center the predictor means
    Z_centered = Z_means - np.mean(Z_means, axis=0)

    V_hat = np.add(V_hat,ph_hat * np.matmul(Z_centered.T, Z_centered))
    
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    
    idx = eigenvalues.argsort()[::-1]  # Get indices that would sort eigenvalues in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    edr_est = solve_triangular(R, eigenvectors)
    # edr_est = solve_triangular(np.sqrt(n_samples) * R, eigenvectors)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    edr_est = edr_est[:, K_index]
    # edr_est = edr_est.flatten()
    if edr_est[0][0] < 0:
        edr_est = -edr_est
    edr_est = edr_est / np.linalg.norm(edr_est)
    
    return edr_est.T, V_hat, R

In [6]:
#sir_modified test on 2*z1t + 3*z2t
import sliced
from sliced import SlicedInverseRegression
num_N = 5
n_obs = 10000
ar_coeff = [0.2, 0.8, 0.5, 0.8]
noise = np.zeros((num_N, n_obs))
n = 100
H = 50
K = 1
hat = 0
Hat = 0
for a in range(n):
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=n_obs)  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs))
    for t in range(0, n_obs):
        ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1]  + noise[0][t]
        ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1]  + noise[1][t]
        ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1]  + noise[2][t]
        ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1]  + noise[3][t]
        ar_series[4][t] = 2*ar_series[0][t] + 3*ar_series[1][t] + noise[4][t]
    y = ar_series[4]
    X = np.concatenate([ar_series[i].reshape(-1,1) for i in range(4)], axis = 1)
    edr, _, R = sir_modified(X, y, H, K=K)
    hat += edr

hat = hat/n
g = hat[0][0]/hat[0][1]
g

0.6663143063279877

In [17]:
#sir_modified test on 2*z1t + 3*z2t
import sliced
from sliced import SlicedInverseRegression
S = 12
num_N = 5
n_obs = 10000
ar_coeff1 = 0.2
ar_coeff2 = 0.8
noise = np.zeros((num_N, n_obs + S))
n = 100
H = 50
K = 1
hat = 0
Hat = 0
for a in range(n):
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=n_obs + S)  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs + S))
    for t in range(0, n_obs + S):
        ar_series[0][t] = ar_coeff1 * ar_series[0][t - 1]  + noise[0][t]
        ar_series[1][t] = ar_coeff2 * ar_series[1][t - 1]  + noise[1][t]
        ar_series[2][t] = 0.3 * ar_series[2][t - 1] + 0.4 * noise[2][t-1] + noise[2][t]
        ar_series[3][t] = -0.4 * noise[3][t-1] + noise[3][t]
        ar_series[4][t] = 2*ar_series[0][t-1] + 3*ar_series[1][t-1] + noise[4][t]
    y = ar_series[4][1:n_obs+1]
    X = np.concatenate([ar_series[i][0:n_obs].reshape(-1,1) for i in range(4)], axis = 1)
    edr, _, R = sir_modified(X, y, H, K=K)
    hat += edr

hat = hat/n
g = hat[0][0]/hat[0][1]
g

0.6669418312654851

In [16]:
#for more complicated SIR for time series examples, use new objective
#sir_modified test on 2*z1t + 3*z2t
import sliced
from sliced import SlicedInverseRegression
S = 12
num_N = 5
n_obs = 10000
ar_coeff1 = 0.2
ar_coeff2 = 0.8
noise = np.zeros((num_N, n_obs + S))
n = 100
H = 50
K = 1
hat = 0
Hat = 0
for a in range(n):
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=n_obs + S)  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs + S))
    for t in range(0, n_obs + S):
        ar_series[0][t] = ar_coeff1 * ar_series[0][t - 1]  + noise[0][t]
        ar_series[1][t] = ar_coeff2 * ar_series[1][t - 1]  + noise[1][t]
        ar_series[2][t] = 0.3 * ar_series[2][t - 1] + 0.4 * noise[2][t-1] + noise[2][t]
        ar_series[3][t] = -0.4 * noise[3][t-1] + noise[3][t]
        ar_series[4][t] = 2*ar_series[0][t-2] + 3*ar_series[1][t-1] + noise[4][t]
    y = ar_series[4][2:n_obs+2]
    X = np.concatenate([ar_series[i][0:n_obs].reshape(-1,1) for i in range(4)], axis = 1)
    edr, _, R = sir_modified(X, y, H, K=K)
    hat += edr

hat = hat/n
g = hat[0][0]/hat[0][1]
g

0.8341902858951119

In [23]:
#from 0 to Q double sum
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
    
def test(ar_coeff):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    n_obs = 10000
    #######
    S = 1
    #######
    noise = np.zeros((num_N, n_obs+S))
    H = 50
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    while n1 < 100:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = 2 * ar_series[0][t] + 3 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            _ , M, Q1 = sir_modified(X, y[a], H, K)
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum(sum((phi[j] ** a) * (V1[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            K_largest_eigenvectors = edr_est[:, K_index]
            # K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += edr_est
        n1 += 1
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = hat[i][0] / hat[i][1]
    array = np.array(hat)
    print(tabulate(array, tablefmt='latex'))
    print(g)

# Example usage
ar_coeff = [0.2, 0.5, 0.2, 0.2]
test(ar_coeff)

\begin{tabular}{rrrr}
\hline
 0.554906 & 0.831899 & 0.000518947 & 0.00016933 \\
\hline
\end{tabular}
[[0.6670353]]


In [28]:
#from 0 to Q double sum
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
    
def test1(ar_coeff):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    n_obs = 10000
    S = 20
    noise = np.zeros((num_N, n_obs+S))
    H = 50
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  
    while n1 < 100:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = 2 * ar_series[0][t] + 3 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V1 = []
        Q2 = []
        for a in range(0, S + 1):
            _ , M, Q1 = sir_modified(X, y[a], H, K)
            V1.append(M)
            Q2.append(Q1)
        for q in range(1, S + 1):
            Q3 = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    #transform back to find V_hat
                    Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T)[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
                    # Q3[j, k] = sum(sum((phi[j] ** a) * (np.linalg.inv(Q2[a]) @ np.linalg.inv(Q2[a]) @ V1[a] @ (np.linalg.inv(Q2[a])).T @ Q2[a])[j, k] * (phi[k] ** a) for a in range(0, l)) for l in range(1, q + 1))
            #Q2[a] @ V[a] @ Q2[a].T by sample covariance matrix
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q3)
            # edr_est = solve_triangular(Q2[0], eigenvectors1)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            # K_largest_eigenvectors = edr_est[:, K_index]
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += edr_est
        n1 += 1
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = hat[i][0] / hat[i][1]
    array = np.array(hat)
    print(tabulate(array, tablefmt='latex'))
    print(g)

# Example usage
ar_coeff = [0.2, 0.5, 0.5, 0.8]
test1(ar_coeff)

\begin{tabular}{rrrr}
\hline
 0.555345 & 0.831611 & 0.000195165 & -0.000383046 \\
 0.547716 & 0.836655 & 0.000228    & -0.000346709 \\
 0.54483  & 0.838537 & 0.000242285 & -0.000327503 \\
 0.543372 & 0.839482 & 0.00024958  & -0.000315886 \\
 0.542497 & 0.840048 & 0.000253931 & -0.00030842  \\
 0.541915 & 0.840424 & 0.000256824 & -0.000303425 \\
 0.541499 & 0.840691 & 0.000258887 & -0.000299868 \\
 0.541188 & 0.840892 & 0.000260431 & -0.000297201 \\
 0.540946 & 0.841047 & 0.000261632 & -0.00029513  \\
 0.540752 & 0.841172 & 0.000262592 & -0.000293476 \\
 0.540594 & 0.841273 & 0.000263376 & -0.000292123 \\
 0.540462 & 0.841358 & 0.00026403  & -0.000290997 \\
 0.540351 & 0.84143  & 0.000264583 & -0.000290044 \\
 0.540255 & 0.841491 & 0.000265057 & -0.000289228 \\
 0.540172 & 0.841544 & 0.000265467 & -0.000288521 \\
 0.5401   & 0.841591 & 0.000265827 & -0.000287902 \\
 0.540036 & 0.841632 & 0.000266143 & -0.000287357 \\
 0.539979 & 0.841668 & 0.000266425 & -0.000286872 \\
 0.539928 & 0.841

In [30]:
ar_coeff = [0.2, 0.3, 0.5, 0.8]
test1(ar_coeff)

\begin{tabular}{rrrr}
\hline
 0.555065 & 0.831795 & 8.99199e-05 & -7.9383e-05  \\
 0.554314 & 0.832296 & 0.000117502 & -9.20674e-05 \\
 0.554055 & 0.832468 & 0.000126326 & -9.68162e-05 \\
 0.553926 & 0.832554 & 0.000130774 & -9.93127e-05 \\
 0.553848 & 0.832606 & 0.000133441 & -0.000100866 \\
 0.553796 & 0.83264  & 0.000135219 & -0.0001019   \\
 0.553759 & 0.832665 & 0.000136488 & -0.000102639 \\
 0.553731 & 0.832683 & 0.00013744  & -0.000103194 \\
 0.55371  & 0.832698 & 0.000138181 & -0.000103626 \\
 0.553693 & 0.832709 & 0.000138773 & -0.000103972 \\
 0.553678 & 0.832719 & 0.000139258 & -0.000104256 \\
 0.553667 & 0.832726 & 0.000139661 & -0.000104492 \\
 0.553657 & 0.832733 & 0.000140003 & -0.000104692 \\
 0.553648 & 0.832739 & 0.000140296 & -0.000104863 \\
 0.553641 & 0.832744 & 0.00014055  & -0.000105011 \\
 0.553634 & 0.832748 & 0.000140772 & -0.000105141 \\
 0.553629 & 0.832752 & 0.000140968 & -0.000105256 \\
 0.553624 & 0.832755 & 0.000141142 & -0.000105358 \\
 0.553619 & 0.832